In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV

### Import the data

In [2]:
main_path = r"C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML/dataSets/"
pd_data_v1 = pd.read_csv(main_path+'dataSet_dron_v1.csv')
pd_data_v4 = pd.read_csv(main_path + 'dataSet_dron_v4_total_salo.csv')

In [3]:
pd_data = pd_data_v4

In [4]:
X = pd_data.drop(columns=["Energy","path_num","points","times"],axis=1)
y = pd_data["Energy"]

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

X_train_mean = x_train.mean(axis=0)
X_train_std = x_train.std(axis=0)

In [6]:
X_train_mean_array = X_train_mean.to_numpy()
X_train_std_array = X_train_std.to_numpy()
x_train_array = x_train.to_numpy()
x_test_array = x_test.to_numpy()

x_train_array = (x_train_array-X_train_mean_array)/X_train_std_array
x_test_array = (x_test_array-X_train_mean_array)/X_train_std_array

In [7]:
columns = x_train.columns
x_train_v1 = pd.DataFrame(data=x_train_array,columns=columns)
x_test_v1 = pd.DataFrame(data=x_test_array,columns=columns)
x_train_v1.head()

,total_dist,Total_time,avg_path_vel,totalTurns
0,1.824554,-0.297957,1.910213,0.296012
1,0.630833,-1.304621,0.837742,0.296012
2,-0.104110,-1.879857,0.122001,-1.080623
3,0.003944,-0.046291,0.011711,0.296012
4,0.705850,0.169423,0.693494,0.296012


### Desarrollo del modelo

In [12]:
param_grid = [
    {'kernel':['poly'],'C':[0.01,0.1,1,10,100,1000],'gamma':['scale','auto'],'degree':[2,3,4,5],'epsilon':[0.001,0.01,0.1,1.0,10.0,100.0]},
    {'kernel':['rbf'],'C':[0.01,0.1,1,10,100,1000],'gamma':['scale','auto'],'epsilon':[0.001,0.01,0.1,1.0,10.0,100.0]},  
    {'kernel':['sigmoid'],'C':[0.01,0.1,1,10,100,1000],'gamma':['scale','auto'],'epsilon':[0.001,0.01,0.1,1.0,10.0,100.0]}, 
    {'kernel':['linear'],'C':[0.01,0.1,1,10,100,1000]}
]

svr = SVR()

svr_model = GridSearchCV(svr,param_grid)

### Regression with MinMax scaler

In [13]:
svr_model.fit(x_train_v1,y_train)

GridSearchCV(estimator=SVR(),
             param_grid=[{'C': [0.01, 0.1, 1, 10, 100, 1000],
                          'degree': [2, 3, 4, 5],
                          'epsilon': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                          'gamma': ['scale', 'auto'], 'kernel': ['poly']},
                         {'C': [0.01, 0.1, 1, 10, 100, 1000],
                          'epsilon': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                          'gamma': ['scale', 'auto'], 'kernel': ['rbf']},
                         {'C': [0.01, 0.1, 1, 10, 100, 1000],
                          'epsilon': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                          'gamma': ['scale', 'auto'], 'kernel': ['sigmoid']},
                         {'C': [0.01, 0.1, 1, 10, 100, 1000],
                          'kernel': ['linear']}])

In [14]:
print(svr_model.best_estimator_)
print(svr_model.best_score_)
print(svr_model.best_params_)

SVR(C=1000, degree=5, epsilon=0.01, kernel='poly')
-0.12174134550386731
{'C': 1000, 'degree': 5, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'poly'}


In [15]:
pred = svr_model.predict(x_test_v1)
RMSE = mean_squared_error(y_test,pred,squared=False)
MSE = mean_squared_error(y_test,pred)
r2 = r2_score(y_test,pred)

print(f'RMSE_minMax: {RMSE}')
print(f'MSE_minMax: {MSE}')
print(f'r2_minMax: {r2}')

RMSE_minMax: 4771236584.646584
MSE_minMax: 2.276469854667e+19
r2_minMax: -0.20582953490250122
